## Imports

In [1]:
from __future__ import annotations

import math
from typing import Literal

from PIL import Image

import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
import torchvision

from torchvision.datasets import FashionMNIST

# pytorch with gpu
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("GPU is available and being used")
else:
    DEVICE = torch.device("cpu")
    print("GPU is not available, using CPU instead")

GPU is available and being used


## Load Computer Vision Dataset

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']